In [ ]:
import os
import requests
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor, as_completed

# -----------------------------------------------------
# CONFIGURATION
# -----------------------------------------------------
xlsm_file = r"GWX"
output_folder = r"C:\Users\GWX-DATA\Documents\Rock 2\Nov25\Images"
sheet_name = "24-30NOV"

# Create folder structure
folders = {
    "adhoc_landmark": os.path.join(output_folder, "adhoc", "landmark"),
    "adhoc_address":  os.path.join(output_folder, "adhoc", "address"),
    "rider_landmark": os.path.join(output_folder, "rider", "landmark"),
    "rider_address":  os.path.join(output_folder, "rider", "address"),
}

for f in folders.values():
    os.makedirs(f, exist_ok=True)

# -----------------------------------------------------
# LOAD WORKBOOK
# -----------------------------------------------------
wb = load_workbook(xlsm_file, data_only=True)
ws = wb[sheet_name]

# -----------------------------------------------------
# FUNCTION TO DOWNLOAD IMAGE
# -----------------------------------------------------
def download_image(url, folder_key, sn):
    """Download image from URL and save to the correct folder with SN."""
    if not url or not isinstance(url, str) or not url.startswith("http"):
        return False  # skip invalid URL

    try:
        resp = requests.get(url, timeout=15)
        if resp.status_code == 200:
            file_path = os.path.join(folders[folder_key], f"{folder_key}_{sn}.jpg")
            with open(file_path, "wb") as f:
                f.write(resp.content)
            return True
        else:
            print(f"⚠️ Failed (status {resp.status_code}) for {folder_key}_{sn}")
            return False
    except Exception as e:
        print(f"⚠️ Error downloading {folder_key}_{sn}: {e}")
        return False

# -----------------------------------------------------
# PREPARE DOWNLOAD TASKS
# -----------------------------------------------------
tasks = []
for row in range(2, ws.max_row + 1):
    vocat = ws[f"M{row}"].value
    if vocat is None:
        continue
    vocat = str(vocat).strip().lower()

    if "adhoc" in vocat:
        base = "adhoc"
    elif "rider" in vocat:
        base = "rider"
    else:
        continue

    row_sn = row - 1  # consistent SN

    # URLs
    landmark_url = ws[f"Q{row}"].value
    address_url  = ws[f"R{row}"].value

    if landmark_url:
        tasks.append((landmark_url, f"{base}_landmark", row_sn))
    if address_url:
        tasks.append((address_url, f"{base}_address", row_sn))

# -----------------------------------------------------
# MULTITHREADED DOWNLOAD
# -----------------------------------------------------
max_threads = 10  # adjust based on your internet speed
with ThreadPoolExecutor(max_workers=max_threads) as executor:
    futures = [executor.submit(download_image, url, key, sn) for url, key, sn in tasks]
    for future in as_completed(futures):
        pass  # optionally, handle completed tasks

print("✅ Finished downloading all images.")
